In [1]:
import os, csv
from dotenv import load_dotenv
import pandas as pd
import time

from datetime import datetime
from xAPIConnector import *
import schedule
import requests

import calendar
import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
XTB_USER_ID = os.getenv('XTB_USER_ID')
XTB_USER_PASSWORD = os.getenv('XTB_USER_PASSWORD')

API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

ts = TimeSeries(key=API_KEYS, output_format='pandas')
# XTB_USER_ID, XTB_USER_PASSWORD
# client = APIClient()

In [3]:
def get_initial_bars(pair, tf, ts):
    bars, _ = ts.get_intraday(symbol=pair, interval=tf, outputsize='full')
    df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with strategy

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.iloc[::-1] # reverse row
    df.reset_index(drop=True, inplace=True)

    return df

In [4]:
def get_latest_bars(df, pair, tf, ts):
    bars, _ = ts.get_intraday(symbol=pair,interval=tf, outputsize='compact')
    incoming_df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with rsi strategy

    incoming_df['timestamp'] = pd.to_datetime(incoming_df['timestamp'], unit='ms')
     
    if (incoming_df['timestamp'].iloc[0] != df['timestamp'].iloc[-1]):
        df = df[:900].append(incoming_df.head(1), ignore_index = True)
    return df

In [5]:
def notifyMsg(*msg):
    r = requests.post(LINE_URL, headers=LINE_HEADERS, data={'message': msg[0]})
    print(r.text)

In [6]:
class RSI_O2_Bot:
    def __init__(self, params):
        self.client = None
        self.pair = params['pair']
        
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.current_action = 'close'
        self.atr = params['atr']
        self.rsi = params['rsi']
        
        self.ema = params['ema']
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        self.pip = params['pip']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        
        self.lot_size = 0.01
        self.order_id = 0
        self.target = 10
        self.round = params['round']
        
        self.df = None
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def calLotSize(self, entry, exit):
        self.lot_size = abs(round(self.target / (((entry - exit) / self.pip ) * self.pip_value ) , 2))
        print(f'lot_size: {str(self.lot_size)}')
        
    def executeOrder(self, action, entry=0, exit=0, sl=0, tp=0):
        self.client = APIClient(port=5112)

        loginResponse = self.client.execute(loginCommand(userId=XTB_USER_ID, password=XTB_USER_PASSWORD))
        logger.info(str(loginResponse)) 

        # check if user logged in correctly
        if(loginResponse['status'] == False):
            print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))
        
        now = calendar.timegm(time.gmtime()) * 100
        if action == 'buy':
            order_action='buy' # 'buy stop'
            tick_resp = self.client.execute(getTickPricesCommand(self.pair, now))
            if tick_resp['returnData']['quotations'][0]['ask'] > round(entry, self.round):
                order_action='buy limit'
            print(f"{order_action} price: {str(round(entry, self.round))}, \
                  ask: {str(tick_resp['returnData']['quotations'][0]['ask'])}, \
                  bid: {str(tick_resp['returnData']['quotations'][0]['bid'])}")
                
#             sl = tick_resp['returnData']['quotations'][0]['ask'] - (entry - sl)
#             tp = tick_resp['returnData']['quotations'][0]['ask'] + (tp - entry)
                
            order_resp = self.client.execute(newPendingOrderCommand(
                symbol=self.pair, 
                action=order_action, 
                price=round(entry, self.round), # tick_resp['returnData']['quotations'][0]['ask'] ,
                tp=round(tp, self.round), 
                sl=round(sl, self.round), 
                volume=self.lot_size
            ))
            self.order_id = order_resp['returnData']['order']
            
        elif action == 'sell':
            order_action='sell' # 'sell stop'
            tick_resp = self.client.execute(getTickPricesCommand(self.pair, now))
            if tick_resp['returnData']['quotations'][0]['bid'] < round(entry, self.round):
                order_action='sell limit'
                
            print(f"{order_action} price: {str(round(entry, self.round))}, \
                  ask: {str(tick_resp['returnData']['quotations'][0]['ask'])}, \
                  bid: {str(tick_resp['returnData']['quotations'][0]['bid'])}")
                
#             sl = tick_resp['returnData']['quotations'][0]['bid'] + (sl - entry)
#             tp = tick_resp['returnData']['quotations'][0]['bid'] - (entry - tp)
                
            order_resp = self.client.execute(newPendingOrderCommand(
                symbol=self.pair, 
                action= order_action, 
                price=round(entry, self.round), # tick_resp['returnData']['quotations'][0]['bid'],
                tp=round(tp, self.round), 
                sl=round(sl, self.round), 
                volume=self.lot_size
            ))
            self.order_id = order_resp['returnData']['order']
            
        elif action == 'close buy':
            order_resp = client.execute(stopOrderCommand(
                orderId=self.order_id,
                symbol=self.pair, 
                action='buy', 
                price=round(exit, self.round),
                volume=self.lot_size
            ))
            
        elif action == 'close sell':
            order_resp = client.execute(stopOrderCommand(
                orderId=self.order_id,
                symbol=self.pair, 
                action='sell', 
                price=round(exit, self.round),
                volume=self.lot_size
            ))
            
        self.client.disconnect()
        
    def do_strategy(self, bars):
        self.df = bars
        rsi_o2 = ta.Strategy(
            name = 'RSI 14 Days for opposite trade',
            description = 'RSI for alert',
            ta = [
                {'kind': 'ema', 'length': self.ema},
                {'kind': 'rsi', 'length': self.rsi},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(rsi_o2)
    
        row = self.df.tail(1).to_dict('records')[0]
        
        if self.current_action == 'close' or self.order == {}:
            if (row['EMA_'+str(self.ema)] < row['low']) and (row['RSI_'+str(self.rsi)] > self.overbought):
#                 row['action'] = 'buy'
                self.order['action'] = 'buy'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr)]
                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'buy'
                notifyMsg(f"Buy {self.pair} RSI_O2\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('buy', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
                
            elif (row['EMA_'+str(self.ema)] > row['high']) and (row['RSI_'+str(self.rsi)] < self.oversold):   
#                 row['action'] = 'sell'
                self.order['action'] = 'sell'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['high'] + row['ATRr_'+str(self.atr)]
                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'sell'
                notifyMsg(f"Sell {self.pair} RSI_O2\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])

        if (self.current_action == 'buy'):
            if (row['low'] <= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                notifyMsg(f"S/L Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
                self.reset_order()
                
            elif (row['high'] >= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                notifyMsg(f"T/P Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()

        elif (self.current_action == 'sell'):
            if (row['high'] >= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                notifyMsg(f"S/L Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()
                
            elif (row['low'] <= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                notifyMsg(f"T/P Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()

In [7]:
class Psar_Bot:
    def __init__(self, params):
        self.client = None
        self.pair = params['pair']
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        self.psar = params['psar']
        
        self.pip = params['pip']
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        
        self.lot_size = 0.01
        self.order_id = 0
        self.target = 10
        self.round = params['round']
        
        self.df = None
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def calLotSize(self, entry, exit):
        self.lot_size = abs(round(self.target / (((entry - exit) / self.pip ) * self.pip_value ) , 2))
        print(f'lot_size: {str(self.lot_size)}')
        
    def executeOrder(self, action, entry=0, exit=0, sl=0, tp=0):
        self.client = APIClient(port=5112)

        loginResponse = self.client.execute(loginCommand(userId=XTB_USER_ID, password=XTB_USER_PASSWORD))
        logger.info(str(loginResponse)) 

        # check if user logged in correctly
        if(loginResponse['status'] == False):
            print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))
        
        now = calendar.timegm(time.gmtime()) * 100
        if action == 'buy':
            order_action='buy' # 'buy stop'
            tick_resp = self.client.execute(getTickPricesCommand(self.pair, now))
            if tick_resp['returnData']['quotations'][0]['ask'] > round(entry, self.round):
                order_action='buy limit'
            print(f"{order_action} price: {str(round(entry, self.round))}, \
                  ask: {str(tick_resp['returnData']['quotations'][0]['ask'])}, \
                  bid: {str(tick_resp['returnData']['quotations'][0]['bid'])}")
                
#             sl = tick_resp['returnData']['quotations'][0]['ask'] - (entry - sl)
#             tp = tick_resp['returnData']['quotations'][0]['ask'] + (tp - entry)
                
            order_resp = self.client.execute(newPendingOrderCommand(
                symbol=self.pair, 
                action=order_action, 
                price=round(entry, self.round), # tick_resp['returnData']['quotations'][0]['ask'] ,
                tp=round(tp, self.round), 
                sl=round(sl, self.round), 
                volume=self.lot_size
            ))
            self.order_id = order_resp['returnData']['order']
            
        elif action == 'sell':
            order_action='sell' # 'sell stop'
            tick_resp = self.client.execute(getTickPricesCommand(self.pair, now))
            if tick_resp['returnData']['quotations'][0]['bid'] < round(entry, self.round):
                order_action='sell limit'
                
            print(f"{order_action} price: {str(round(entry, self.round))}, \
                  ask: {str(tick_resp['returnData']['quotations'][0]['ask'])}, \
                  bid: {str(tick_resp['returnData']['quotations'][0]['bid'])}")
                
#             sl = tick_resp['returnData']['quotations'][0]['bid'] + (sl - entry)
#             tp = tick_resp['returnData']['quotations'][0]['bid'] - (entry - tp)
                
            order_resp = self.client.execute(newPendingOrderCommand(
                symbol=self.pair, 
                action= order_action, 
                price=round(entry, self.round), # tick_resp['returnData']['quotations'][0]['bid'], 
                tp=round(tp, self.round), 
                sl=round(sl, self.round), 
                volume=self.lot_size
            ))
            self.order_id = order_resp['returnData']['order']
            
        elif action == 'close buy':
            order_resp = client.execute(stopOrderCommand(
                orderId=self.order_id,
                symbol=self.pair, 
                action='buy', 
                price=round(exit, self.round),
                volume=self.lot_size
            ))
            
        elif action == 'close sell':
            order_resp = client.execute(stopOrderCommand(
                orderId=self.order_id,
                symbol=self.pair, 
                action='sell', 
                price=round(exit, self.round),
                volume=self.lot_size
            ))
            
        self.client.disconnect()
        
    def do_strategy(self, bars):
        self.df = bars
        parabolic_sar = ta.Strategy(
            name = 'Parabolic Sar',
            description = 'Parabolic Sar Reversal',
            ta = [
                {'kind': 'psar', 'af0': self.psar, 'af': self.psar, 'max_af': self.psar*10},
                {'kind': 'ema', 'length': self.ema},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(parabolic_sar)
    
        row = self.df.tail(1).to_dict('records')[0]
        pre_row = self.df.tail(2).to_dict('records')[0]
        
        if self.current_action == 'close':
                
            if ~np.isnan(row['PSARl_'+str(self.psar)+'_'+str(self.psar*10)]) and \
                ~np.isnan(pre_row['PSARs_'+str(self.psar)+'_'+str(self.psar*10)]) and \
                (row['close'] > row['EMA_'+str(self.ema)]):
                    
                self.order['action'] = 'buy'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr)]
                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'buy'
                notifyMsg(f"Buy {self.pair} Psar\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('buy', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
                
            elif ~np.isnan(row['PSARs_'+str(self.psar)+'_'+str(self.psar*10)]) and \
                ~np.isnan(pre_row['PSARl_'+str(self.psar)+'_'+str(self.psar*10)]) and \
                (row['close'] < row['EMA_'+str(self.ema)]):

                self.order['action'] = 'sell'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['high'] + row['ATRr_'+str(self.atr)]
                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'sell'
                notifyMsg(f"Sell {self.pair} Psar\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])

        if (self.current_action == 'buy'):
            if (row['low'] <= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                notifyMsg(f"S/L Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
                self.reset_order()
                
            elif (row['high'] >= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                notifyMsg(f"T/P Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()

        elif (self.current_action == 'sell'):
            if (row['high'] >= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                notifyMsg(f"S/L Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()
                
            elif (row['low'] <= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                notifyMsg(f"T/P Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()

In [8]:
eurusd_o2 = RSI_O2_Bot({
    'pair': 'EURUSD',
    'atr': 17,
    'rsi': 10,
    'ema': 200,
    'overbought': 73,
    'oversold': 27,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
})

eurusd_psar = Psar_Bot({
    'pair': 'EURUSD',
    'atr': 17,
    'psar': 0.01,
    'ema': 200,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5, 
})

audusd_o2 = RSI_O2_Bot({
    'pair': 'AUDUSD',
    'atr': 19,
    'rsi': 10,
    'ema': 200,
    'overbought': 73,
    'oversold': 27,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
})

audusd_psar = Psar_Bot({
    'pair': 'AUDUSD',
    'atr': 19,
    'psar': 0.01,
    'ema': 200,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5, 
})

In [9]:
eurusd_init_bars = get_initial_bars('EURUSD', tf='15min', ts=ts)
audusd_init_bars = get_initial_bars('AUDUSD', tf='15min', ts=ts)

eurusd_o2_hours = [14, 20, 22, 23]
audusd_o2_hours = [20, 21, 22]
eurusd_psar_hours = [14, 19, 20, 21]
audusd_psar_hours = [11, 18, 20, 21, 22]

In [10]:
def get_all_realtime():
    current_hour = datetime.now().hour
    
    eurusd_bars = get_latest_bars(eurusd_init_bars, pair='EURUSD', tf='15min', ts=ts)
    
    if current_hour in eurusd_o2_hours:
        eurusd_o2.do_strategy(eurusd_bars)
    if current_hour in eurusd_psar_hours:
        eurusd_psar.do_strategy(eurusd_bars)
    
    audusd_bars = get_latest_bars(audusd_init_bars, pair='AUDUSD', tf='15min', ts=ts)
    
    if current_hour in audusd_o2_hours:
        audusd_o2.do_strategy(audusd_bars)
    if current_hour in audusd_psar_hours:
        audusd_psar.do_strategy(audusd_bars)
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-06-17 12:23:09)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

{"status":200,"message":"ok"}
lot_size: 0.03


[2022-06-17 14:08:37,772][connect:74] Socket connected
[2022-06-17 14:08:37,773][_waitingSend:88] Sent: b'{"command": "login", "arguments": {"userId": "1959587", "password": "veGan/1102199c", "appName": ""}}'
[2022-06-17 14:08:38,692][_read:107] Received: {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-0000ae69-001e6119-4079bf70da4c8f19-bc3bef78'}
[2022-06-17 14:08:38,693][executeOrder:39] {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-0000ae69-001e6119-4079bf70da4c8f19-bc3bef78'}
[2022-06-17 14:08:38,695][_waitingSend:88] Sent: b'{"command": "getTickPrices", "arguments": {"level": 0, "symbols": ["EURUSD"], "timestamp": 165544971800}}'
[2022-06-17 14:08:38,958][_read:107] Received: {'status': True, 'returnData': {'quotations': [{'symbol': 'EURUSD', 'ask': 1.05009, 'bid': 1.04995, 'high': 1.05593, 'low': 1.04935, 'askVolume': 350000, 'bidVolume': 350000, 'timestamp': 1655449718569, 'level': 0, 'exemode': 1, 'spreadRaw': 0.00014, 'spreadTable': 1.4}]}}
[2022-06-17 14:08:38,9

buy limit price: 1.05,                   ask: 1.05009,                   bid: 1.04995


[2022-06-17 14:08:39,229][_read:107] Received: {'status': True, 'returnData': {'order': 656731760}}
[2022-06-17 14:08:39,229][close:115] Closing socket


{"status":200,"message":"ok"}
lot_size: 0.03


[2022-06-17 14:08:41,769][connect:74] Socket connected
[2022-06-17 14:08:41,771][_waitingSend:88] Sent: b'{"command": "login", "arguments": {"userId": "1959587", "password": "veGan/1102199c", "appName": ""}}'
[2022-06-17 14:08:42,339][_read:107] Received: {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-0000ae69-001e611f-8819b03f1a4c9eb3-515a6a23'}
[2022-06-17 14:08:42,342][executeOrder:36] {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-0000ae69-001e611f-8819b03f1a4c9eb3-515a6a23'}
[2022-06-17 14:08:42,345][_waitingSend:88] Sent: b'{"command": "getTickPrices", "arguments": {"level": 0, "symbols": ["EURUSD"], "timestamp": 165544972200}}'
[2022-06-17 14:08:42,616][_read:107] Received: {'status': True, 'returnData': {'quotations': [{'symbol': 'EURUSD', 'ask': 1.05011, 'bid': 1.04998, 'high': 1.05593, 'low': 1.04935, 'askVolume': 450000, 'bidVolume': 350000, 'timestamp': 1655449721561, 'level': 0, 'exemode': 1, 'spreadRaw': 0.00013, 'spreadTable': 1.3}]}}
[2022-06-17 14:08:42,6

buy limit price: 1.05,                   ask: 1.05011,                   bid: 1.04998


[2022-06-17 14:08:42,904][_read:107] Received: {'status': True, 'returnData': {'order': 656731906}}
[2022-06-17 14:08:42,905][close:115] Closing socket
